In [19]:
import tensorflow as tf
import numpy as np
import typing
import pandas as pd
import seaborn as sns
from tensorflow import keras
from hydra import compose, initialize
from omegaconf import OmegaConf
from musicnet.config.Config import Config
from musicnet.preprocessing.utils import get_datasets_info, load_vocabs
from musicnet.preprocessing.wav_chunks_tfrecord.utils import create_tf_record_ds
from musicnet.models.utils import MODEL_PATH
from musicnet.preprocessing.dataset.base import DsName
from matplotlib import pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from musicnet.preprocessing.utils import load_source_dataset, get_datasets_info, load_vocabs
import mido

In [2]:
initialize(version_base=None, config_path="../scripts")
cfg = compose(config_name="defaults", overrides=["stages=[]", "exp=False"])

In [5]:
config = typing.cast(Config, OmegaConf.to_object(cfg))
ds_infos = get_datasets_info(config)

In [6]:
ds_infos

[DsInfo(name='train', config=DsConfig(wav_source=SynthMidiToWavConfig(type=<WavSourceType.SYNTH_MIDI_TO_WAV: 'synth_midi_to_wav'>, tps=100, min_note_ticks=1, max_note_ticks=100, max_silmultaneous_notes=9, sidechannel_sparsity=1, min_note=20, max_note=105, notes_std=20, track_length_per_note=900, velocity_min=64, velocity_max=127), preprocessor=WavChunksTFRecordPreprocessorConfig(type=<PreprocessorType.WAV_CHUNKS_TFRECORDS: 'wav_chunks_tfrecords'>, params=WavChunksTFRecordPreprocessorParams(chunk_size_sec=10, chunk_shift_sec=5, target_sr=44100, note_rounding=0.01, spectogram=SpectogramParams(n_fft=2205, hop_length=441, min_hz=0, n_filters=200, unit=<SpectogramUnit.DECIBELS: 'decibels'>)), file_count=20), batch_size=8, split=None)),
 DsInfo(name='val', config=DsConfig(wav_source=SynthMidiToWavConfig(type=<WavSourceType.SYNTH_MIDI_TO_WAV: 'synth_midi_to_wav'>, tps=100, min_note_ticks=1, max_note_ticks=100, max_silmultaneous_notes=9, sidechannel_sparsity=1, min_note=20, max_note=105, notes

In [7]:
dataset = load_source_dataset(ds_infos[-1].config, ds_infos[-1].src_name)

In [12]:
track = dataset.get_track(1728)

In [17]:
track.get_notes()

,note,channel,instrument,start,end,velocity
0,64,4,1,1.437500,1.631925,100.0
1,61,4,1,1.437500,1.631925,100.0
2,57,4,1,1.437500,1.631925,100.0
3,52,4,1,1.437500,1.631925,100.0
4,49,4,1,1.437500,1.631925,100.0
...,...,...,...,...,...,...
2737,85,4,1,236.886908,236.886908,100.0
2738,81,4,1,236.886908,236.886908,100.0
2739,76,4,1,236.886908,236.886908,100.0
2740,73,4,1,236.886908,236.886908,100.0


In [14]:
track.get_notes()["end"].max()

236.8869084583339

In [15]:
track.get_duration()

251.26167800453516

In [20]:
midi_path = track.get_midi_path()
midi = mido.MidiFile(midi_path)

In [21]:
midi.ticks_per_beat

480

In [26]:
time = 0
for msg in midi:
    time += msg.time
print(time)

251.26638146250053
